# BeautyfulSoup 네이버 블로그 스크래핑

## 블로그 포스트 스크래핑 함수 정의

In [14]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_posts(query, page_num = 10):
    url_query = quote(query)    # 인코딩8
    url = 'https://search.naver.com/search.naver?where=post&query=' + url_query
    post_df = pd.DataFrame(columns = ('Title','date','blogger','bolg url','post url','post'))
    idx = 0
    
    for _ in range(0, page_num):

        search_url = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(search_url, 'html.parser')

        posts = soup.find_all('li', {'class':'sh_blog_top'})

        for post in posts:
            title = post.find('a', {'class' : 'sh_blog_title _sp_each_url _sp_each_title'}).get('title')
            date = post.find('dd', {'class' : 'txt_inline'}).get_text()
            blogger = post.find('a',{'class': 'txt84'}).get_text()
            blog_url = post.find('a',{'class': 'txt84'}).get('href')
            post_url = post.find('a', {'class' : 'url'}).get('href')    #이건 link니까 get_text하면 안됨
            # 이건 post의 url
            post_link = urllib.request.urlopen(post_url).read()
            post_html = BeautifulSoup(post_link, 'html.parser')
            # 실제적인 블로거의 url

            for main_frame in post_html.select('iframe#mainFrame'):
                frame_url = 'https://blog.naver.com' + main_frame.get('src')
                post_text = urllib.request.urlopen(frame_url).read()
                post_html = BeautifulSoup(post_text, 'html.parser')
                post_content_text = ''

                for post_content in post_html.find_all('div', {'class' : 'se-main-container'}):
                    post_content_text = post_content.get_text()

                post_content_text = post_content_text.replace('\n', '')
                post_content_text = post_content_text.replace('\t', '')


                post_df.loc[idx] = [title,date,blogger, blog_url,post_url, post_content_text]
                idx += 1

                print('#',end = '')

        try:
            next = soup.find('a', {'class': 'next'}).get('href')
            url = 'https://search.naver.com/search.naver' +next
        except:
            break
    return post_df



In [16]:
query = input('검색 : ')
post_df = get_posts(query,5)
print('Done')

검색 : 청하
#####################################Done


## 블로그 포스트 스크래핑 실행

In [13]:
post_df

,Title,date,blogger,bolg url,post url,post
0,"포항 체리체험,드로잉체험 카페 청하 체리힐/포항 월포해수욕장 드로잉체험 카페 청하 체리힐",2020.07.09.,여행을조아하고스타...,https://blog.naver.com/angeljung486,https://blog.naver.com/angeljung486?Redirect=L...,
1,[음악리뷰] 청하 'PLAY' 청하의 여름이 돌아왔다.,2020.07.06.,문현곱창 블로그,https://blog.naver.com/404506,https://blog.naver.com/404506?Redirect=Log&log...,청하가 두번째 선공개곡 플레이를 6일 오후 공개 했습니다.스테이 투나잇과 달리 이번...
2,청하 음영메이크업 돌체앤가바나 뷰티 아이섀도우 팔레트로,2020.05.22.,쎌리의 시간,https://blog.naver.com/sh_theme,https://blog.naver.com/sh_theme?Redirect=Log&l...,​​우아하면서 고혹적인 컨셉의 청하 stay toninght 신곡 무대 보셨나요솔로...
3,청하부르는 날씨,2020.06.25.,mon ami 의 생활기록...,https://blog.naver.com/0416109,https://blog.naver.com/0416109?Redirect=Log&lo...,
4,하이닉스 배석종스시 청하와 잘 어울리는 맛,3일 전,라쏘스윗,https://blog.naver.com/ra_sso,https://blog.naver.com/ra_sso?Redirect=Log&log...,"​​ 이천 하이닉스 배석종스시,광어랑 연어가 맛있는 초밥집!!​​며칠 전부터 초밥이..."
5,"윌슨아니고이빨, 청하에메로나, 와이리시끄럽노!, 집에갈레, 심장에해로운쪼꼬미들, 참치사랑",2일 전,bok_soong2,https://blog.naver.com/dokyung1120,https://blog.naver.com/dokyung1120?Redirect=Lo...,​I Know It's Not Easy but You're Not Alone An...
6,#159. 장범준_청하 <벌써 12시>_Acoustic Cover Ver.,2020.08.16.,Vinomad's Winessay...,https://blog.naver.com/avantcentre,https://blog.naver.com/avantcentre?Redirect=Lo...,벌써 12시 아티스트청하발매일2019.01.02. Lyrics Cf. 가사는 원곡과...
7,20200502먹방’ 전지적참견시점_청하_닭볶음탕맛집[유림보신원]마시쪙,2020.06.13.,Love never failsෆ,https://blog.naver.com/0420bjw,https://blog.naver.com/0420bjw?Redirect=Log&lo...,​​ 전지적참견시점_청하_닭볶음탕유림​ 유림보신원서울한복판인데 계곡에서 먹는 느낌의...
8,청하체리힐 : 포항체리농장,2020.06.25.,mood kiki,https://blog.naver.com/shuri_kiki,https://blog.naver.com/shuri_kiki?Redirect=Log...,​​​​​​​​​​컨디션좋았던 주말!배도 든든하게 채웠고친한동생이랑 전화하드만 촌에...
9,돌체앤가바나 뷰티 화보 속 청하 메이크업! 컬러 마스카라&아이섀도우 팔레트 추천해요,2020.05.22.,썸블리 뷰티시크릿,https://blog.naver.com/pparksomin,https://blog.naver.com/pparksomin?Redirect=Log...,
